# Excess Chemical Potential of a Salt Solution: Widom Particle Insertion Method

## Table of Contents
* [Introduction](#Introduction)
* [Widom Particle Insertion Method](#Widom-Particle-Insertion-Method)
* [Simulation Setup](#Simulation-Setup)
* [Production Run and Comparison with Analytical Theory](#Production-Run-and-Comparison-with-Analytical-Theory)
* [Further Exercises](#Further-Exercises)
* [References](#References)

## Introduction

The combination of Monte Carlo and Molecular Dynamic simulations offers many possibilities beyond using each technique solely on their own.
In general, MD simulations are restricted to relatively small time scales and, depending on the modeled interactions, to rather small particle numbers as well.
However, they have the advantage of computing collective particle movements with the correct dynamics.
The time between two MD steps is bound by the fastest motion of the system of interest (for all-atom MD, those are atomic vibrations).

MC simulations, on the other hand, do not have such limitations.
Even nonphysical moves can be considered, still leading to results with correct thermodynamic properties.
However, no valuable information of the system's dynamics can be obtained.
The combination of MD and MC moves allows us to mimic any ensemble.

In this tutorial, you will compute the chemical potential of a monovalent salt solution by using a combination of MD and MC techniques.
The chemical potential is important for simulating chemical equilibria and systems where particles can be exchanged with an external reservoir, which is too large to be simulated explicitly, or between two subsystems.

## Widom Particle Insertion Method

In a pure MD simulation, there is no direct way to access the chemical potential, since it is related to changes in the particle number, which is fixed in an MD simulation.
In 1963, Widom proposed a MC scheme to measure the chemical potential in a system using trial particle insertions <a href='#[1]'>[1]</a>.

The chemical potential for species $\alpha$ is defined as
\begin{equation}
  \mu_{\alpha} = \left(\frac{\partial G}{\partial N_\alpha} \right)_{p,T,N_{\beta\neq\alpha}} = \left(\frac{\partial F}{\partial N_\alpha}\right)_{V,T,N_{\beta\neq\alpha}} \quad .\tag{1}
  \label{eq:chem_pot_def}
\end{equation}
It corresponds to the difference in the Helmholtz free energy $F$ or Gibbs free energy $G$ for bringing a particle from infinity into the system, or equally for removing one particle, thus changing the particle number $N_\alpha$.

The canonical partition function is defined as
\begin{equation}
  Z(N,V,T)=\frac{V^N}{\Lambda^{3N}N!} \int_{\left[0,1\right]^{3N}}\exp\left(-\beta U\left(\left\{\boldsymbol{s}^i\right\}\right)\right)\,\mathrm{d}\boldsymbol{s}^1\ldots\mathrm{d}\boldsymbol{s}^N  \quad ,\tag{2}
  \label{eq:partition_function}
\end{equation}
where V is the volume, $\Lambda=h/\sqrt{2\pi m k_\mathrm{B}T}$ the thermal wavelength, $\beta$ the inverse temperature, $U$ the interaction potential and $\boldsymbol{s}^i$ the rescaled (dimensionless) particle coordinates.

The Helmholtz free energy $F$ can be computed from the partition function: $F=-k_BT\ln\left(Z(N,V,T)\right)$.
For sufficiently large N, the following relation approximates the chemical potential:
\begin{align}
  \mu & = -k_\mathrm{B}T \ln\left(\frac{Z(N+1,V,T)}{Z(N,V,T)}\right) \label{eq:mu_limit_largeN}\\
      & = \mu_\textrm{ideal}+\mu_\textrm{ex} \nonumber\\
      & = - k_\mathrm{B}T\ln\left(\frac{V}{\Lambda^{3}(N+1)}\right)+\mu_\textrm{ex}\, , \nonumber\tag{3}
\end{align}
where $\mu_\textrm{ideal}$ depends only on the density of the system, so it is a constant for systems with different interactions but the same particle density.
The interesting part is the excess chemical potential $\mu_\textrm{ex}$, which can be expressed in a form involving a canonical average:
\begin{equation}
  \mu-\mu_\textrm{ideal}=\mu_\textrm{ex}=-k_\mathrm{B}T\ln\left(\int_{\left[0,1\right]^{3}}\langle\exp(-\beta\Delta U)\rangle\,\mathrm{d}\boldsymbol{s}^{N+1}\right)\quad .\tag{4}
  \label{eq:mu_ex}
\end{equation}
 
In the above expression, $\Delta U$ is the energy difference of the system before and after the insertion of a test particle at a random position.
The value of the integral in this equation can be computed in a simulation in the canonical ensemble (e.g. using Langevin dynamics) by averaging over many configurations of the system (corresponding to the average $\langle ...\rangle$) and many positions of the trial insertion (corresponding to the integral).

The system under consideration here consists of $N$ monovalent ion pairs in an implicit solvent with $\epsilon_\text{r} = 78$, which is a simple model (``restricted primitive model'') for an aqueous salt solution of e.g. NaCl.
In order to avoid that particles overlap, the Weeks-Chandler-Andersen potential is used.
In the unit system employed in the simulations, the following parameters will be used:

\begin{align*}
\lambda_b=2.0\,,\quad k_BT=1.0\,,\quad z_i=1.0\,,\quad N=N_i=50
\end{align*}

Because the investigated system is a monovalent salt the concentrations (and thus, densities) of positive and negative ions are the same.
The method of Widom insertion introduced can be used to calculate the excess chemical potential for this system.
However, we can not calculate the excess chemical potential of a single ion (a quantitity that is also not accessible in experiments) but only that of an ion pair, which is given by
\begin{align}
 \mu_\textrm{ex}^\textrm{pair} = \mu_\textrm{ex}^+ + \mu_\textrm{ex}^- =
  \mu-\mu_\textrm{ideal}=\mu_\textrm{ex}=-k_BT\ln\left(\int_{\left[0,1\right]^{6}}\langle\exp(-\beta\Delta U)\rangle \mathrm{d}\boldsymbol{s}_{+}^{N+1}\mathrm{d}\boldsymbol{s}_{-}^{N+1}\right).\tag{5}
\end{align}
Always inserting ion pairs ensures the overall electroneutrality of the system and prevents a diverging electrostatic energy.


# Simulation Setup

In a first step, we import the required **ESPResSo** features as well as some additional python modules:

In [21]:
import espressomd
import espressomd.electrostatics
from espressomd import reaction_methods

espressomd.assert_features(['ELECTROSTATICS', 'P3M', 'WCA'])

import tqdm
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(42)
plt.rcParams.update({'font.size': 18})

Next, we define the simulation units and set the parameters which define our system and the interactions:

In [22]:
# units
# we assume a unit system where the elementary charge, the thermal energy and the WCA length scale (sigma) are all 1
sigma = 3.55e-10 # Sigma in SI units
avo = 6.022e+23 # Avogadro's number in SI units
PREF = 1/(10**3 * avo * sigma**3) # Prefactor to mol/L
BJERRUM_LENGTH = 2.0

# Weeks-Chandler-Andersen interaction
LJ_EPSILON = 1.0
LJ_SIGMA = 1.0

# Langevin-Thermostat
KT = 1.0
GAMMA = 1.0

# number of salt ion pairs
N_ION_PAIRS = 50

# particle types and charges
types = {
    "Xplus": 1, # Positive ions
    "Xminus": 2, # Negative ions
}

charges = {
    "Xplus": 1.0,
    "Xminus": -1.0,
}

Now we are ready to set up the system. Because we will need to rescale the simulation box, we will initially only set up the short-range interactions.

**Exercise:**
* Set up a system with box length $10\,\sigma$, integrator time step $\Delta t=0.01 \,\tau$ ($\tau$ is the Lennard-Jones timescale, i.e. equal to 1 in the employed unit system) and Verlet skin width of $\delta = 0.4\sigma$
* Add a total of ``N_ION_PAIRS`` ion pairs at random positions to the simulation box
* Add a WCA interaction (purely repulsive Lennard-Jones interaction) between the particles

**Hints:**
* Refer to the documentation to find out how to set up the WCA interaction ([Non-bonded Interactions](https://espressomd.github.io/doc/inter_non-bonded.html))

```python
system = espressomd.System(box_l=3 * [10.0])

system.time_step = 0.01
system.cell_system.skin = 0.4

system.part.add(type=[types["Xplus"]]*N_ION_PAIRS, pos=np.random.rand(N_ION_PAIRS, 3) * 20.0, q = [charges["Xplus"]]*N_ION_PAIRS)
system.part.add(type=[types["Xminus"]]*N_ION_PAIRS, pos=np.random.rand(N_ION_PAIRS, 3) * 20.0, q = [charges["Xminus"]]*N_ION_PAIRS)

for i in types:
    for j in types:
        system.non_bonded_inter[types[i], types[j]].wca.set_params(epsilon=LJ_EPSILON, sigma=LJ_SIGMA)
```

**Exercise:**
* Implement a function ``system_setup(c_salt_SI) `` that takes the desired salt concentration ``c_salt_SI`` in SI-units (mol/l) as an argument and rescales the simulation box accordingly
* Afterwards, the function should also add the P3M actor to account for electrostatic interactions

**Hints:**
* Use the prefactor ``PREF `` defined above to convert the concentration from SI-units to the unit system employed in the simulation
* An accuracy of $10^{-3}$ for the P$^3$M should be enough for this exercise

```python
def system_setup(c_salt_SI):
    print("Salt concentration (in mol/l):", c_salt_SI)
    c_salt_sim = c_salt_SI / PREF
    box_length = np.power(N_ION_PAIRS / c_salt_sim, 1.0/3.0)
    
    # rescale box
    system.change_volume_and_rescale_particles(d_new=box_length)
    print("Rescaled the simulation box.")
    
    # add P3M
    p3m = espressomd.electrostatics.P3M(prefactor=BJERRUM_LENGTH, accuracy=1e-3)
    system.actors.add(p3m)
    p3m_params = p3m.get_params()
```

**Exercise:**
* Implement a function ``warmup `` that removes potential overlaps between particles
* After the overlaps have been removed, the integrator should be switched to Velocity-Verlet (with Langevin thermostat) in order to perform a warmup integration

``` python
def warmup():
    system.integrator.set_steepest_descent(f_max=0, gamma=1e-3, max_displacement=0.01)
    
    print("Removing overlaps...")
    for i in range(100):
        system.integrator.run(100)
        
    system.integrator.set_vv()
    system.thermostat.set_langevin(kT=KT, gamma=GAMMA, seed=42)
    
    print("Running warmup integration...")
    for i in range(100):
        system.integrator.run(1000)
```

**Exercise:**
* Add the functionality to perform Widom insertion moves. Make sure that you always insert an ion pair in order to conserve the electroneutrality of the system.


**Hints:**
* Refer to the documentation to find out how to set up Widom particle insertion ([Widom Insertion](https://https://espressomd.github.io/doc/reaction_methods.html#widom-insertion-for-homogeneous-systems))

```python
# add reaction for Widom particle insertion
widom = espressomd.reaction_methods.WidomInsertion(kT=KT, seed=42)
widom.add_reaction(reactant_types=[], reactant_coefficients=[], product_types=[types["Xplus"], types["Xminus"]], product_coefficients=[1, 1], default_charges={types["Xplus"]: charges["Xplus"], types["Xminus"]: charges["Xminus"]})
widom.set_non_interacting_type(type=len(types)+1)
```

**Exercise:**
* Implement a function ``calculate_excess_chemical_potential `` that performs Widom particle insertion and calculates the chemical potential at the end
* After the sampling loop, the function should remove all actors and turn off the thermostat

```python
def calculate_excess_chemical_potential():
    potential_energy_samples = []
    
    print("Sampling...")
    for i in tqdm.tqdm(range(500)):
        system.integrator.run(100)
        
        for j in range(100):
           potential_energy_samples.append(widom.calculate_particle_insertion_potential_energy(reaction_id=0)) 
        
    excess_chemical_potential, error_excess_chemical_potential = widom.calculate_excess_chemical_potential(particle_insertion_potential_energy_samples=potential_energy_samples)
    print("Done. Excess chemical potential:", excess_chemical_potential, "\n")
    
    system.actors.clear()
    system.thermostat.turn_off()
    return excess_chemical_potential, error_excess_chemical_potential
```

# Production Run and Comparison with Analytical Theory

Now we are ready to perform measurements of the excess chemical potential using the Widom particle insertion method.

**Exercise:**
* Perform chemical potential measurements using the Widom method at different salt concentrations $c_{\text{salt}}\in\left\{0.001\,\mathrm{M},\, 0.003\,\mathrm{M},\,0.01\,\mathrm{M},\, 0.03\,\mathrm{M},\,0.1\,\mathrm{M},\, 0.3\,\mathrm{M}\right\}$
* Plot the measured excess chemical potential as a function of $c_{\text{salt}}$.
* Explain the observed behaviour qualitatively
* How do you expect the excess chemical potential to behave in the limit $c_{\text{salt}}\rightarrow 0$?

**Hints:**
* The plot of the excess chemical potential is best viewed on a logarithmically scaled $x$-axis

```python
salt_concentrations = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3]
excess_chemical_potentials = []
import time
start = time.time()
for c_salt_SI in salt_concentrations:
    system_setup(c_salt_SI)
    warmup()
    excess_chemical_potentials.append(calculate_excess_chemical_potential())
end = time.time()
print("Elapsed time:", end-start)
```

```python
fig = plt.figure(figsize=(10, 8))
plt.errorbar(salt_concentrations, np.asarray(excess_chemical_potentials)[:,0], yerr=np.asarray(excess_chemical_potentials)[:,1], linestyle="none", marker="s", markersize=10.0, label=r"Widom Method")
plt.xlabel(r"salt concentration $c_\mathrm{{salt}}$ in mol/l")
plt.ylabel("excess chemical potential $\mu_\mathrm{{ex}}/k_\mathrm{{B}}T$")
plt.xscale("log")
plt.xlim((6e-4, 0.6))
plt.ylim((-0.8, 0.1))
plt.legend()
plt.show()
```

There exists a number of analytical and phenomenological theories to describe 
the excess chemical potential of salt solutions using analytical expressions. Examples are the Debye-Hückel (DH) equation,
\begin{equation}
\mu_\textrm{ex}^\textrm{DH}=-2\cdot k_\mathrm{B}T\cdot 1.17 \frac{\sqrt{c_\mathrm{{salt}}/1\,\mathrm{M}}}{1+\sqrt{c_\mathrm{{salt}}/1\,\mathrm{M}}}
\end{equation}
and the Davies equation,
\begin{equation}
\mu_\textrm{ex}^\textrm{Davies}=-2\cdot k_\mathrm{B}T\cdot 1.17 \left(\frac{\sqrt{c_\mathrm{{salt}}/1\,\mathrm{M}}}{1+\sqrt{c_\mathrm{{salt}}/1\,\mathrm{M}}}-0.2\cdot c_\mathrm{{salt}}/1\,\mathrm{M}\right)
\end{equation}
which results from an empirical extension of the Debye-Hückel theory.

**Exercise:**
* Compare your simulation results with the Debye-Hückel theory and the Davies equation by plotting all three curves in a single plot

```python
def davies_equation(ionic_strength):
    A = 0.51
    mu_ex = - 2 * A * (np.sqrt(ionic_strength) / (1 + np.sqrt(ionic_strength)) - 0.2 * ionic_strength) / np.log10(np.exp(1))
    return mu_ex

def dh_equation(ionic_strength):
    A = 0.51
    mu_ex = - 2 * A * (np.sqrt(ionic_strength) / (1 + np.sqrt(ionic_strength))) / np.log10(np.exp(1))
    return mu_ex

concentration_range = np.logspace(-4, 0.0, num=500, base=10)

fig = plt.figure(figsize=(10, 8))
plt.plot(concentration_range, davies_equation(concentration_range), label=r"Davies equation", color="r")
plt.plot(concentration_range, dh_equation(concentration_range), label=r"Debye-Hückel equation", color="black", linestyle="dashed")
plt.errorbar(salt_concentrations, np.asarray(excess_chemical_potentials)[:,0], yerr=np.asarray(excess_chemical_potentials)[:,1], linestyle="none", marker="s", markersize=10.0, label=r"Widom Method")
plt.xlabel(r"salt concentration $c_\mathrm{{salt}}$ in mol/l")
plt.ylabel("excess chemical potential $\mu_\mathrm{{ex}}/k_\mathrm{{B}}T$")
plt.xscale("log")
plt.xlim((6e-4, 0.6))
plt.ylim((-0.8, 0.1))
plt.legend()
plt.show()
```

# Further Exercises

1. Derive equation (4) starting from the canonical partition function.
2. What problems would you run into if you tried to use the Widom insertion method with large molecules (e.g. polymers) as test particles?

# References

<a id='[1]'></a>[1] B. Widom. Some topics in the theory of fluids. Journal of Chemical Physics, 39:2808,
1963.